# File Connector

The FileConnector gives agents tools to operate on files: create, read, edit, search, and delete. All operations go through the workspace sandbox, so the agent sees `/workspace/...` paths while files live in isolated host directories.

In [ ]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.connectors.file import FileConnector
from agentic_patterns.core.user_session import set_user_session

## Setup and Wrap FileConnector as Agent Tools

Set the user/session context once, then wrap FileConnector methods as agent tools. No need to pass context through -- it's resolved automatically from contextvars.

In [ ]:
set_user_session("demo", "file_connector_demo")

connector = FileConnector()

tools = [
    connector.append,
    connector.delete,
    connector.edit,
    connector.find,
    connector.head,
    connector.list,
    connector.read,
    connector.write,
]

## Agent Creates and Edits a File

We ask the agent to create a markdown file, then modify it. The agent decides which tools to call and in what order.

In [3]:
agent = get_agent(tools=tools)

prompt = """Do the following steps:
1. Create a file /workspace/notes.md with a title '# Meeting Notes' and three bullet points about a project kickoff meeting.
2. Read the file back to verify it was created.
3. Edit line 3 to change the second bullet point to '- Agreed on weekly sync every Monday'.
4. Read the file again and show me the final content."""

agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print(f"\nFinal answer: {agent_run.result.output}")

Agent step: UserPromptNode(user_prompt="Do the following steps:\n1. Create a file /workspace/notes.md with a title 
'# Meeting Notes' and three bullet points about a project kickoff meeting.\n2. Read the file back to verify it was 
created.\n3. Edit line 3 to change the second bullet point to '- Agreed on weekly sync every Monday'.\n4. Read the 
file again and show me the final content.", instructions_functions=[], system_prompts=(), 
system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content="Do the following steps:\n1. Create
a file /workspace/notes.md with a title '# Meeting Notes' and three bullet points about a project kickoff 
meeting.\n2. Read the file back to verify it was created.\n3. Edit line 3 to change the second bullet point to '- 
Agreed on weekly sync every Monday'.\n4. Read the file again and show me the final content.", 
timestamp=datetime.datetime(2026, 1, 31, 14, 22, 43, 936071, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="I'll help you complete these steps.
Let me start by creating the file, then reading and editing it as requested."), 
ToolCallPart(tool_name='file_write', args='{"path": "/workspace/notes.md", "content": "# Meeting Notes\\n- 
Discussed project goals and timeline\\n- Assigned team roles and responsibilities\\n- Reviewed budget and resource 
allocation"}', tool_call_id='toolu_vrtx_015J8WoMSECEjsa8vpJ63aDE')], usage=RequestUsage(input_tokens=1225, 
output_tokens=126, details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 1, 31, 14, 22, 47, 899897, tzinfo=datetime.timezone.utc), provider_name='openai',
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 1, 31, 14, 22, 45, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1769869365-QZyX9uCQ8LYtCq1Q5DtI', finish_reason='tool_call', 
run_id='51b5c762-2f52-4c90-be14-f139c0af308c'))

Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='file_write', content='Wrote 139 
bytes to /workspace/notes.md', tool_call_id='toolu_vrtx_015J8WoMSECEjsa8vpJ63aDE', 
timestamp=datetime.datetime(2026, 1, 31, 14, 22, 47, 985137, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='file_read', args='{"path": 
"/workspace/notes.md"}', tool_call_id='toolu_vrtx_01UDuM5LP7Sf2wemrhUbc5q7')], 
usage=RequestUsage(input_tokens=1376, output_tokens=58, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 1, 31, 14, 22, 49, 578128, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'tool_calls', 'timestamp': datetime.datetime(2026, 1, 31, 14, 22, 48, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1769869368-nXSnzg5wGvD36M4FXGcX', finish_reason='tool_call', 
run_id='51b5c762-2f52-4c90-be14-f139c0af308c'))

...

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='file_read', args='{"path": 
"/workspace/notes.md"}', tool_call_id='toolu_vrtx_01GD8XxnnkrWECeX6z5Y4T5J')], 
usage=RequestUsage(input_tokens=1631, output_tokens=58, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 1, 31, 14, 22, 53, 199112, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'tool_calls', 'timestamp': datetime.datetime(2026, 1, 31, 14, 22, 51, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1769869371-9THgua3AOv6Mg9inuAFo', finish_reason='tool_call', 
run_id='51b5c762-2f52-4c90-be14-f139c0af308c'))

Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='file_read', content='# Meeting 
Notes\n- Discussed project goals and timeline\n- Agreed on weekly sync every Monday\n- Reviewed budget and resource
allocation\n', tool_call_id='toolu_vrtx_01GD8XxnnkrWECeX6z5Y4T5J', timestamp=datetime.datetime(2026, 1, 31, 14, 22,
53, 215542, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='Perfect! All steps completed 
successfully. Here\'s a summary:\n\n1. ✅ **Created** `/workspace/notes.md` with a title and three bullet points 
about a project kickoff meeting\n2. ✅ **Verified** the file was created correctly\n3. ✅ **Edited** line 3 to 
change the second bullet point to "Agreed on weekly sync every Monday"\n4. ✅ **Final content:**\n\n```markdown\n# 
Meeting Notes\n- Discussed project goals and timeline\n- Agreed on weekly sync every Monday\n- Reviewed budget and 
resource allocation\n```\n\nThe file has been successfully created and edited as requested!')], 
usage=RequestUsage(input_tokens=1729, output_tokens=149, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 1, 31, 14, 22, 56, 562237, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 1, 31, 14, 22, 53, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1769869373-DgUHD8oxuoZdRV4OCtsl', finish_reason='stop', 
run_id='51b5c762-2f52-4c90-be14-f139c0af308c'))

Agent step: End(data=FinalResult(output='Perfect! All steps completed successfully. Here\'s a summary:\n\n1. ✅ 
**Created** `/workspace/notes.md` with a title and three bullet points about a project kickoff meeting\n2. ✅ 
**Verified** the file was created correctly\n3. ✅ **Edited** line 3 to change the second bullet point to "Agreed 
on weekly sync every Monday"\n4. ✅ **Final content:**\n\n```markdown\n# Meeting Notes\n- Discussed project goals 
and timeline\n- Agreed on weekly sync every Monday\n- Reviewed budget and resource allocation\n```\n\nThe file has 
been successfully created and edited as requested!'))


Final answer: Perfect! All steps completed successfully. Here's a summary:

1. ✅ **Created** `/workspace/notes.md` with a title and three bullet points about a project kickoff meeting
2. ✅ **Verified** the file was created correctly
3. ✅ **Edited** line 3 to change the second bullet point to "Agreed on weekly sync every Monday"
4. ✅ **Final content:**

```markdown
# Meeting Notes
- Discussed project goals and timeline
- Agreed on weekly sync every Monday
- Reviewed budget and resource allocation
```

The file has been successfully created and edited as requested!


## Verify on Disk

The file exists in the host filesystem under the user's workspace directory.

In [ ]:
from pathlib import PurePosixPath
from agentic_patterns.core.workspace import container_to_host_path

host_path = container_to_host_path(PurePosixPath("/workspace/notes.md"))
print(f"Host path: {host_path}")
print(f"Content:\n{host_path.read_text()}")